In [ ]:
# !pip install chromadb sentence_transformers

In [2]:
import tarfile

# 압축 해제
print("압축을 해제하는 중...")
with tarfile.open("chroma_db.tar.gz", "r:gz") as tar:
    tar.extractall(path=".")  # 현재 위치에 풀기

print("완료! 'chroma_db' 폴더가 생성되었습니다.")

압축을 해제하는 중...


C:\Users\kouls\AppData\Local\Temp\ipykernel_34376\1336457635.py:6: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=".")  # 현재 위치에 풀기


완료! 'chroma_db' 폴더가 생성되었습니다.


In [3]:
import chromadb
from sentence_transformers import SentenceTransformer
import torch

# ---------------------------------------------------------
# 1. 설정 (GPU 확인 등)
# ---------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ---------------------------------------------------------
# 2. 모델 로드 (저장할 때 썼던 그 모델!)
# ---------------------------------------------------------
print("모델 로드 중...")
model = SentenceTransformer("dragonkue/BGE-m3-ko").to(device)

# ---------------------------------------------------------
# 3. ChromaDB 연결 (다운로드 받은 폴더 경로 지정)
# ---------------------------------------------------------
# path="./chroma_db" 는 압축 푼 폴더 이름과 같아야 합니다.
client = chromadb.PersistentClient(path="./chroma_db")

# 컬렉션 가져오기 (create가 아니라 get_collection 사용)
collection = client.get_collection(name="patent_claims")

print(f"✅ 데이터베이스 로드 완료! 총 데이터 수: {collection.count()}개")



c:\Users\kouls\anaconda3\envs\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
모델 로드 중...


c:\Users\kouls\anaconda3\envs\llm_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kouls\.cache\huggingface\hub\models--dragonkue--BGE-m3-ko. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP d

✅ 데이터베이스 로드 완료! 총 데이터 수: 589049개


In [ ]:
# ---------------------------------------------------------
# 4. 검색 함수 정의
# ---------------------------------------------------------
def search_patents(query_text, top_k=10):
    # 1) 검색어를 벡터로 변환 (임베딩)
    query_vector = model.encode(query_text).tolist()

    # 2) 벡터로 DB 검색
    results = collection.query(
        query_embeddings=[query_vector],
        n_results=top_k
    )

    # 3) 결과 출력
    print(f"\n🔍 검색어: '{query_text}'")
    
    for i in range(top_k):
        doc = results['documents'][0][i]
        meta = results['metadatas'][0][i]
        distance = results['distances'][0][i]
        
        print(f"--- [Rank {i+1}] (거리: {distance:.4f}) ---")
        print(f"특허명: {meta.get('title', '제목없음')}")
        print(f"특허ID: {meta.get('patent_id')} (Claim {meta.get('claim_no')})")
        print(f"내용: {doc[:100]}...") # 100자만 미리보기

# ---------------------------------------------------------
# 5. 실제 검색 테스트
# ---------------------------------------------------------
search_patents("수집된 영상을 컴퓨터비전 프로그램의 필터를 통해 영상을 얼굴 영역과 흉부 영역을 구분")


🔍 검색어: '수집된 영상을 컴퓨터비전 프로그램의 필터를 통해 영상을 얼굴 영역과 흉부 영역을 구분'
--- [Rank 1] (거리: 0.2656) ---
특허명: 가상현실을 통한 시니어 이용자의 심리진단 및 치유 시스템
특허ID: 1020220161550 (Claim 1)
내용: 피험자의 영상을 수집하는 영상 획득 모듈과;수집된 영상을 컴퓨터비전 프로그램의 필터를 통해 영상을 얼굴 영역과 흉부 영역을 구분하고, 얼굴 영역의 미세표정을 인식하며, 피험자의 호...
--- [Rank 2] (거리: 0.3026) ---
특허명: 가상현실을 통한 시니어 이용자의 심리진단 및 치유 시스템
특허ID: 1020220161550 (Claim 2)
내용: 상기 데이터 처리 모듈은,수집된 영상을 컴퓨터비전 프로그램의 cascade classifier를 통해 얼굴 영역과 흉부 영역을 구분하여 추출하는 ROI 분할부와;얼굴인식 프로그램을...
--- [Rank 3] (거리: 0.3883) ---
특허명: 2D 카메라 얼굴 영상기반 비접촉형 심박수 측정 방식의 정확도 향상을 위한 ROI 추출 최적화 연구
특허ID: 1020230030465 (Claim 1)
내용: 하나 이상의 프로세서들, 및상기 하나 이상의 프로세서들에 의해 실행되는 하나 이상의 프로그램들을 저장하는 메모리를 구비한 컴퓨팅 장치에서 수행되는 방법으로서, 얼굴 영상을 획득하는...
--- [Rank 4] (거리: 0.3929) ---
특허명: 영상 처리 방법 및 장치
특허ID: 1020200093297 (Claim 1)
내용: 사용자의 얼굴을 포함하는 영상 프레임을 획득하는 단계;상기 영상 프레임에서 상기 사용자의 얼굴 영역을 검출하는 단계; 상기 얼굴 영역 중 적어도 일부에 가려짐(occlusion)이...
--- [Rank 5] (거리: 0.3940) ---
특허명: 차량 탑승자의 표정 인식 장치 및 방법
특허ID: 1020230180920 (Claim 11)
내용: 상기 얼굴 영역이 포함된 입력 영상을 준비하

In [5]:
def group_results_by_patent(results):
    counts = {}
    metas = results["metadatas"][0]

    for m in metas:
        pat_id = m["patent_id"]
        counts[pat_id] = counts.get(pat_id, 0) + 1

    return counts

In [6]:
# 1. 쿼리 텍스트 정의
query = "편광판을 이용한 액정"

# 2. 데이터를 저장했던 *동일한 모델*로 쿼리를 직접 임베딩
#    (이때 model 변수가 이전에 로드되어 있어야 합니다)
query_emb = model.encode(query).tolist()

# 3. query_texts 대신 query_embeddings 파라미터 사용
results = collection.query(query_embeddings=[query_emb], n_results=10)

# 4. 이후 로직은 동일
patent_hit_map = group_results_by_patent(results)
print("특허별 히트 수:", patent_hit_map)

특허별 히트 수: {'1020247042361': 1, '1020160153262': 1, '1020170024706': 2, '1020247013771': 1, '1020257027309': 1, '1020237001747': 1, '1020250024294': 1, '1020220037297': 1, '1020257023376': 1}


In [7]:
results

{'ids': [['1020247042361_claim8',
   '1020160153262_claim5',
   '1020170024706_claim6',
   '1020247013771_claim4',
   '1020257027309_claim10',
   '1020237001747_claim3',
   '1020250024294_claim2',
   '1020220037297_claim7',
   '1020170024706_claim1',
   '1020257023376_claim7']],
 'embeddings': None,
 'documents': [['제 6 항에 기재한 편광판을 갖는 액정 표시 장치.',
   '액정 셀의 적어도 한쪽의 면에, 제1항 또는 제2항에 기재된 편광판이 배치된 액정 패널.',
   '제1항 내지 제3항 중 어느 한 항에 기재한 편광판 세트와 액정 셀을 구비하고, 상기 액정 셀의 두께가 0.4 mm 이하인 액정 패널.',
   '상기 산성 용액이, 염산, 황산 또는 질산인, 편광판의 제조 방법.',
   '상기 액정 배향막이, 편광 자외선을 조사하여 얻어지는, 액정 배향막.',
   '제1항 또는 상기 편광자층이, 중합성 액정 화합물과 상기 2색성 유기 색소를 함유하는 것을 특징으로 하는 편광판.',
   '편광판용인, 접착제 조성물.',
   '편광판인, 광학 부재.',
   '액정 셀의 시인측에 배치되는 제1 편광판과, 상기 액정 셀의 배면측에 배치되는 제2 편광판을 갖는 편광판 세트로서, 상기 제2 편광판은 반사형 편광 필름을 갖고, 상기 제1 편광판을 80℃에서 4시간 유지했을 때의 흡수축 방향에 있어서의 폭 2 mm당 수축력을 F1로 하고, 투과축 방향에 있어서의 폭 2 mm당 수축력을 F2로 하며, 상기 제2 편광판을 80℃에서 4시간 유지했을 때의 흡수축 방향에 있어서의 폭 2 mm당 수축력을 F3으로 하고, 투과축 방향에 있어서의 폭 2 mm당 수축력을 F4로 했을 때, (F1×F2)/(F3×F4)가 0.5∼